# TP03 — Inverse Kinematics (Session 1: Exercises 2-4)

**GEII3 — Industrie 4.0: Robotique | Lab Session 3 - Part 1**

Continuation of IK methods: algebraic solutions and numerical approaches.

---

In [ ]:
# Setup (copy from main notebook or re-run)
import numpy as np
import matplotlib.pyplot as plt
from math import cos, sin, pi, atan2, sqrt, acos, asin, atan
from scipy.optimize import fsolve, minimize, least_squares
from scipy.linalg import pinv, norm

try:
    import roboticstoolbox as rtb
    from spatialmath import SE3
    RTB_AVAILABLE = True
except ImportError:
    RTB_AVAILABLE = False

# Import helper functions
from tp_02_fk_helpers import (
    fk_dh, fk_ets, rotx, roty, rotz, transx, transy, transz,
    extract_position, extract_rotation
)

from tp_03_ik_helpers import (
    plot_robot_2d, compare_solutions, normalize_angle,
    create_rtb_robot_from_dh, create_rtb_robot_from_ets
)

np.set_printoptions(precision=4, suppress=True)
print("✓ Setup complete")

---

# Exercise 2: 3R Planar Robot - Algebraic IK

**Estimated time**: 60 minutes

## Problem Description

3-DOF planar robot with:
- $L_1 = 0.3$ m
- $L_2 = 0.3$ m  
- $L_3 = 0.2$ m

**Task**: Given target pose $(x_d, y_d, \phi_d)$ where $\phi_d$ is end-effector orientation, find $(q_1, q_2, q_3)$.

## Task 2.1: Geometric Approach with Decoupling (25 min)

**Key insight**: The wrist position is:

$$\begin{aligned}
x_w &= x_d - L_3 \cos(\phi_d) \\
y_w &= y_d - L_3 \sin(\phi_d)
\end{aligned}$$

### Your derivation:

**Step 1**: Compute wrist position $(x_w, y_w)$ using $L_3$ and $\phi_d$

**Step 2**: Use 2R IK to find $(q_1, q_2)$ to reach wrist

**Step 3**: Compute $q_3$ from orientation constraint:
$$\phi_d = q_1 + q_2 + q_3$$
$$q_3 = \phi_d - q_1 - q_2$$

---

## Task 2.2: Implementation (35 min)

In [ ]:
# Robot parameters
L1_3r = 0.3
L2_3r = 0.3
L3_3r = 0.2

dh_3r_planar = [
    (0, 0, L1_3r, 0, 'r'),
    (0, 0, L2_3r, 0, 'r'),
    (0, 0, L3_3r, 0, 'r'),
]

def ik_3r_planar(x_d, y_d, phi_d, L1, L2, L3, elbow='up'):
    """
    Analytical IK for 3R planar robot with orientation control.
    
    Parameters:
    - x_d, y_d: desired end-effector position
    - phi_d: desired end-effector orientation (radians)
    - L1, L2, L3: link lengths
    - elbow: 'up' or 'down'
    
    Returns:
    - q: [q1, q2, q3] if solution exists, None otherwise
    """
    # TODO: Implement based on derivation
    
    # Step 1: Compute wrist position
    x_w = x_d - L3 * cos(phi_d)
    y_w = y_d - L3 * sin(phi_d)
    
    print(f"  Wrist position: ({x_w:.4f}, {y_w:.4f})")
    
    # Step 2: Solve 2R problem for wrist
    D = sqrt(x_w**2 + y_w**2)
    
    # Check reachability
    if D > (L1 + L2) or D < abs(L1 - L2):
        print(f"  Wrist unreachable! D={D:.3f}")
        return None
    
    # Solve for q2
    cos_q2 = (x_w**2 + y_w**2 - L1**2 - L2**2) / (2 * L1 * L2)
    cos_q2 = np.clip(cos_q2, -1, 1)
    
    q2 = acos(cos_q2) if elbow == 'up' else -acos(cos_q2)
    
    # Solve for q1
    alpha = atan2(y_w, x_w)
    beta = atan2(L2 * sin(q2), L1 + L2 * cos(q2))
    q1 = alpha - beta
    
    # Step 3: Solve for q3 from orientation
    q3 = phi_d - q1 - q2
    
    # Normalize angles
    q1 = np.arctan2(np.sin(q1), np.cos(q1))
    q2 = np.arctan2(np.sin(q2), np.cos(q2))
    q3 = np.arctan2(np.sin(q3), np.cos(q3))
    
    return np.array([q1, q2, q3])


# Test cases
print("\n" + "="*70)
print("  3R PLANAR ROBOT - ANALYTICAL IK")
print("="*70)

test_poses = [
    (0.4, 0.3, np.deg2rad(30), "Moderate pose"),
    (0.5, 0.0, np.deg2rad(0), "Horizontal reach"),
    (0.3, 0.4, np.deg2rad(-45), "Vertical orientation"),
]

for x_d, y_d, phi_d, desc in test_poses:
    print(f"\n[Target: x={x_d}, y={y_d}, φ={np.rad2deg(phi_d):.1f}° - {desc}]")
    
    # Solve for both elbow configs
    for elbow in ['up', 'down']:
        q = ik_3r_planar(x_d, y_d, phi_d, L1_3r, L2_3r, L3_3r, elbow=elbow)
        
        if q is not None:
            print(f"  Elbow {elbow:5s}: q = {np.rad2deg(q).astype(int)}°")
            
            # Verify with FK
            T, _ = fk_dh(dh_3r_planar, q)
            pos = extract_position(T)
            
            # Extract orientation (sum of angles for planar)
            phi_reached = np.sum(q)
            
            error_pos = np.linalg.norm([pos[0] - x_d, pos[1] - y_d])
            error_ori = abs(phi_reached - phi_d)
            
            print(f"                Pos error: {error_pos:.2e} m")
            print(f"                Ori error: {np.rad2deg(error_ori):.2e}°")

# Visualize one solution
q_test = ik_3r_planar(0.4, 0.3, np.deg2rad(30), L1_3r, L2_3r, L3_3r, 'up')
if q_test is not None:
    _, T_list = fk_dh(dh_3r_planar, q_test)
    
    fig, ax = plt.subplots(figsize=(8, 8))
    plot_robot_2d(T_list, ax, 
                  title=f"3R Planar - IK Solution\nq={np.rad2deg(q_test).astype(int)}°",
                  target=(0.4, 0.3))
    
    # Draw desired orientation
    phi_d = np.deg2rad(30)
    ax.arrow(0.4, 0.3, 0.1*cos(phi_d), 0.1*sin(phi_d),
             head_width=0.02, head_length=0.015, fc='magenta', 
             ec='magenta', linewidth=2, label='Desired orientation')
    ax.legend()
    plt.show()

---

# Exercise 3: Numerical IK Methods

**Estimated time**: 70 minutes

## Objective

Implement and compare numerical IK methods:
1. Newton-Raphson (Jacobian pseudo-inverse)
2. Damped Least Squares
3. Scipy optimization

## Task 3.1: Jacobian Pseudo-Inverse Method (25 min)

**Algorithm**:

```
Given: T_desired, q_initial
Repeat until convergence:
    1. T_current = FK(q)
    2. e = pose_error(T_current, T_desired)
    3. J = numerical_jacobian(q)
    4. Δq = pinv(J) · e
    5. q = q + α·Δq  (α = step size)
    6. If ||e|| < tolerance: break
```

---

## Task 3.2: Implementation Newton-Raphson (25 min)

In [ ]:
def ik_newton_raphson(dh_params, T_target, q_init, max_iter=100, tol=1e-4, alpha=0.5):
    """
    Numerical IK using Newton-Raphson with Jacobian pseudo-inverse.
    
    Parameters:
    - dh_params: DH parameter table
    - T_target: desired end-effector transformation (4x4)
    - q_init: initial joint configuration
    - max_iter: maximum iterations
    - tol: convergence tolerance
    - alpha: step size (0 < alpha <= 1)
    
    Returns:
    - q: solution joint angles
    - converged: True if converged
    - iterations: number of iterations
    - history: error history
    """
    q = q_init.copy()
    history = []
    
    for iteration in range(max_iter):
        # Forward kinematics
        T_current, _ = fk_dh(dh_params, q)
        
        # Compute error
        error = pose_error(T_current, T_target)
        error_norm = np.linalg.norm(error)
        history.append(error_norm)
        
        # Check convergence
        if error_norm < tol:
            return q, True, iteration + 1, history
        
        # Compute Jacobian
        J = numerical_jacobian(dh_params, q)
        
        # Pseudo-inverse
        J_pinv = pinv(J)
        
        # Update
        delta_q = J_pinv @ error
        q = q + alpha * delta_q
        
        # Normalize angles
        q = np.array([np.arctan2(np.sin(qi), np.cos(qi)) for qi in q])
    
    return q, False, max_iter, history


# Test with 3R planar
print("\n" + "="*70)
print("  NUMERICAL IK - NEWTON-RAPHSON")
print("="*70)

# Target pose
x_t, y_t, phi_t = 0.4, 0.3, np.deg2rad(30)
T_target = np.eye(4)
T_target[0, 3] = x_t
T_target[1, 3] = y_t
T_target[:2, :2] = np.array([[cos(phi_t), -sin(phi_t)],
                              [sin(phi_t),  cos(phi_t)]])

# Initial guess (random)
q_init = np.array([0.5, 0.3, -0.2])

print(f"\nTarget: x={x_t}, y={y_t}, φ={np.rad2deg(phi_t):.1f}°")
print(f"Initial guess: {np.rad2deg(q_init).astype(int)}°")

# Solve
q_sol, converged, iters, history = ik_newton_raphson(
    dh_3r_planar, T_target, q_init, max_iter=50, alpha=0.3
)

print(f"\nConverged: {converged}")
print(f"Iterations: {iters}")
print(f"Solution: {np.rad2deg(q_sol).astype(int)}°")
print(f"Final error: {history[-1]:.2e}")

# Verify
T_reached, _ = fk_dh(dh_3r_planar, q_sol)
pos_reached = extract_position(T_reached)
print(f"\nReached position: ({pos_reached[0]:.4f}, {pos_reached[1]:.4f})")
print(f"Position error: {np.linalg.norm(pos_reached[:2] - [x_t, y_t]):.2e} m")

# Plot convergence
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Error history
ax1.plot(history, 'b-', linewidth=2)
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Error norm')
ax1.set_title('Newton-Raphson Convergence')
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# Robot configuration
_, T_list = fk_dh(dh_3r_planar, q_sol)
plot_robot_2d(T_list, ax2, 
              title=f"Solution\nq={np.rad2deg(q_sol).astype(int)}°",
              target=(x_t, y_t))

plt.tight_layout()
plt.show()

## Task 3.3: Damped Least Squares (20 min)

Improves behavior near singularities by adding damping.

In [ ]:
def ik_damped_ls(dh_params, T_target, q_init, max_iter=100, tol=1e-4, 
                 alpha=0.5, lambda_damp=0.01):
    """
    Damped Least Squares IK.
    
    Uses: Δq = (J^T J + λ I)^(-1) J^T e
    """
    q = q_init.copy()
    n = len(q)
    history = []
    
    for iteration in range(max_iter):
        T_current, _ = fk_dh(dh_params, q)
        error = pose_error(T_current, T_target)
        error_norm = np.linalg.norm(error)
        history.append(error_norm)
        
        if error_norm < tol:
            return q, True, iteration + 1, history
        
        J = numerical_jacobian(dh_params, q)
        
        # Damped pseudo-inverse
        JTJ = J.T @ J
        damped_inv = np.linalg.inv(JTJ + lambda_damp * np.eye(n))
        delta_q = damped_inv @ J.T @ error
        
        q = q + alpha * delta_q
        q = np.array([np.arctan2(np.sin(qi), np.cos(qi)) for qi in q])
    
    return q, False, max_iter, history


print("\n" + "="*70)
print("  DAMPED LEAST SQUARES IK")
print("="*70)

# Same target
q_sol_dls, conv_dls, iters_dls, hist_dls = ik_damped_ls(
    dh_3r_planar, T_target, q_init, max_iter=50, alpha=0.3, lambda_damp=0.01
)

print(f"\nDLS Solution: {np.rad2deg(q_sol_dls).astype(int)}°")
print(f"Converged: {conv_dls}, Iterations: {iters_dls}")
print(f"Final error: {hist_dls[-1]:.2e}")

# Compare convergence
plt.figure(figsize=(10, 6))
plt.plot(history, 'b-', label='Newton-Raphson', linewidth=2)
plt.plot(hist_dls, 'r--', label='Damped LS', linewidth=2)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Error norm', fontsize=12)
plt.title('Convergence Comparison', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

---

# Exercise 4: 3R Anthropomorphic - Decoupling Method

**Estimated time**: 40 minutes

## Problem Description

3-DOF spatial robot (from TP02):
- $d_1 = 0.4$ m
- $a_2 = 0.5$ m
- $a_3 = 0.4$ m

**Task**: Position-only IK (3 DOF for 3D position)

## Task 4.1: Geometric Solution (40 min)

**Approach**:
1. $q_1$: Project target onto XY plane, use atan2
2. $(q_2, q_3)$: 2R problem in vertical plane

---

In [ ]:
# Robot parameters
d1_anthro = 0.4
a2_anthro = 0.5
a3_anthro = 0.4

dh_3r_anthro = [
    (0, d1_anthro, 0,        pi/2, 'r'),
    (0, 0,         a2_anthro, 0,   'r'),
    (0, 0,         a3_anthro, 0,   'r'),
]

def ik_3r_anthropomorphic(x_d, y_d, z_d, d1, a2, a3, elbow='up'):
    """
    Analytical IK for 3R anthropomorphic (position only).
    
    Returns: [q1, q2, q3] or None
    """
    # Joint 1: rotation to align with target in XY plane
    q1 = atan2(y_d, x_d)
    
    # Project to vertical plane
    r = sqrt(x_d**2 + y_d**2)  # Radial distance in XY
    z_eff = z_d - d1            # Height above shoulder
    
    # 2R problem in (r, z_eff) plane
    D = sqrt(r**2 + z_eff**2)
    
    # Check reachability
    if D > (a2 + a3) or D < abs(a2 - a3):
        print(f"  Target unreachable! D={D:.3f}, range=[{abs(a2-a3):.3f}, {a2+a3:.3f}]")
        return None
    
    # Solve for q3
    cos_q3 = (r**2 + z_eff**2 - a2**2 - a3**2) / (2 * a2 * a3)
    cos_q3 = np.clip(cos_q3, -1, 1)
    q3 = acos(cos_q3) if elbow == 'up' else -acos(cos_q3)
    
    # Solve for q2
    alpha = atan2(z_eff, r)
    beta = atan2(a3 * sin(q3), a2 + a3 * cos(q3))
    q2 = alpha - beta
    
    return np.array([q1, q2, q3])


# Test
print("\n" + "="*70)
print("  3R ANTHROPOMORPHIC - ANALYTICAL IK")
print("="*70)

targets_3d = [
    (0.3, 0.3, 0.8, "Moderate reach"),
    (0.0, 0.5, 1.0, "Forward reach"),
    (0.4, 0.0, 0.5, "Low position"),
]

for x_d, y_d, z_d, desc in targets_3d:
    print(f"\n[Target: ({x_d}, {y_d}, {z_d}) - {desc}]")
    
    for elbow in ['up', 'down']:
        q = ik_3r_anthropomorphic(x_d, y_d, z_d, d1_anthro, a2_anthro, a3_anthro, elbow)
        
        if q is not None:
            print(f"  Elbow {elbow:5s}: q = {np.rad2deg(q).astype(int)}°")
            
            # Verify
            T, _ = fk_dh(dh_3r_anthro, q)
            pos = extract_position(T)
            error = np.linalg.norm(pos - [x_d, y_d, z_d])
            print(f"                Error: {error:.2e} m")

print("\n✓ Exercise 4 complete")
print("\n--- End of Session 1 ---\n")
print("Next: Session 2 with PUMA 560, trajectory tracking, and advanced topics")